In [1]:
import tensorflow as tf
import chromadb
from sentence_transformers import SentenceTransformer
import ollama
import fitz

c:\Users\Philemon\anaconda3\envs\transform\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="documents")

# Load an embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Sample documents to add
documents = [
    {"id": "doc1", "text": "Quantum computing is the future of cryptography."},
    {"id": "doc2", "text": "Neural networks are the foundation of deep learning."},
    {"id": "doc3", "text": "The Large Hadron Collider is the world's most powerful particle accelerator."}
]

# Convert text into embeddings and store in Chroma
for doc in documents:
    embedding = embed_model.encode(doc["text"]).tolist()
    collection.add(ids=[doc["id"]], embeddings=[embedding], metadatas=[{"text": doc["text"]}])

print("Documents added to ChromaDB!")


Add of existing embedding ID: doc1
Add of existing embedding ID: doc2
Add of existing embedding ID: doc3
Add of existing embedding ID: doc1
Insert of existing embedding ID: doc1
Add of existing embedding ID: doc2
Insert of existing embedding ID: doc2
Add of existing embedding ID: doc3
Insert of existing embedding ID: doc3


Documents added to ChromaDB!


In [3]:
query = "What is deep learning?"
query_embedding = embed_model.encode(query).tolist()

results = collection.query(query_embedding, n_results=2)


for doc in results["metadatas"][0]:
    print(f"Relevant doc: {doc['text']}")

Relevant doc: Neural networks are the foundation of deep learning.
Relevant doc: The Large Hadron Collider is the world's most powerful particle accelerator.


In [4]:
# Generate prompt with retrieved documents
retrieved_docs = "\n".join([doc["text"] for doc in results["metadatas"][0]])
prompt = f"Use the following information to answer the question:\n{retrieved_docs}\n\nQuestion: {query}"

# Send to Ollama
response = ollama.chat(model="gemma3:1b", messages=[{"role": "user", "content": prompt}])

print("Ollama's Response:", response["message"]["content"])

Ollama's Response: The provided information doesn't explain what deep learning is. It simply states that neural networks are the foundation of deep learning. 

To answer the question "What is deep learning?" we'd need more context. 

**Therefore, I cannot answer the question based on the given information.**



In [5]:
doc = fitz.open("Mensah Philemon's CV.pdf")
text = ""
for page in doc:
    text += page.get_text()
document = {"id": "doc4", "text": text}
embedding = embed_model.encode(document["text"]).tolist()
collection.add(ids=[document["id"]], embeddings=[embedding], metadatas=[{"text": document["text"]}])
print("Added successfully")

Add of existing embedding ID: doc4
Insert of existing embedding ID: doc4


Added successfully


In [6]:
query = "What is the name of the student?"
query_embedding = embed_model.encode(query).tolist()

result = collection.query(query_embedding, n_results=1)
retrieved = result["metadatas"][0][0]["text"]
prompt = f"Use the following information to answer the question:\n{retrieved}\n\nQuestion: {query}"
response = ollama.chat(model="gemma3:1b", messages=[{"role": "user", "content": prompt}])
print("Ollama's Response:", response["message"]["content"])

Ollama's Response: The name of the student is Mensah Philemon Edem Yao.
